# Docker BRATS results

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import os
import seaborn as sb
import math
import pickle
from pprint import pprint
from nltk import agreement
from sklearn.metrics import cohen_kappa_score
%matplotlib inline

In [81]:
# set up the variables and environment
met = 'wt'
# stupid switch for dumb experiments to make data pretty
stupid = False

#set baseline variables for export and stuff
res = 400 # dpi value for image export
form = 'png' #image format for matplotlib exports
filename = met + '_dice_scores.csv'
#rename colums to their 3 character IDs
candidates = {
        'aju' : 'tumor_istb_aj_class.nii',
        'aca' : 'data_data_prediction.nii.gz',
        'kch' : 'tumor_qtimlab_class.nii.gz',
        'ekr' : 'tumor_00000000_class.nii.gz',
        'aka' : 'tumor_kamleshp_class.nii.gz',
        'mag' : 'tumor_magnrbm_class.nii',
        'sse' : 'tumor_saras_tb_class.nii.gz',
        'rsa': 'tumor_gevaertlab_class.nii',
        'gwa' : 'brats_dc_brats2016_test_klhd_pat101_3.nii.gz',
        'ise' : 'tumor_brats2017_isensee_class.nii.gz',
        'mav' : 'majvote_fusion.nii.gz',
        'sim' : 'simple_fusion.nii.gz',
        'sim2': 'simple2_fusion.nii.gz',
        'none' : 'default'
    }

sseRep = {
    'brats2016_test_cbica_patAMQ_362' : 0,
    'brats2016_test_tcia_pat457_0079' : 0,
    'brats2016_test_cbica_patAOQ1_1' : 0
}
#invert the indices to assign the proper labels
inv_cand = {v: k for k, v in candidates.items()}

irrScores = {}

In [82]:
#import data
data = pandas.read_csv(filename)
#data

In [83]:
#preprocess data
def preprocessor(data, met):
    # remove every 2nd row (unnecessary column heads)
    clean = data.replace({'tumor_saras_tb_class.nii.gz' : sseRep})
    clean = clean.iloc[::2]
    # remove patient name column
    # clean = clean.drop('patient', axis=1)
    # rename columns to 3 digit ID
    clean = clean.rename(inv_cand, axis=1)
    # store csv
    #clean.reset_index(inplace=True)
    #print(clean)
    clean.to_csv(met+'_clean.csv')
    # read a clean copy of the preprocessed csv again
    data = pandas.read_csv('fixed_data/'+met+'_clean.csv')
    # remove wrong row indices
    data = data.drop('Unnamed: 0', axis=1)
    return data

In [84]:
data = preprocessor(data, met)
# read a clean copy of the preprocessed csv again
# data = pandas.read_csv(met+'_clean.csv')
# remove wrong row indices
# data = data.drop('Unnamed: 0', axis=1)

### Inter Rater Reliability

In [85]:
# IRR function using the Cohen Kappa Score 
# https://en.wikipedia.org/wiki/Cohen%27s_kappa
# create empty dataframe for correlation indices

def irr(data, res, base=5):
    """ Data is the Dataframe with the initial data
    irr is an empty DataFrame with matching columns and indices
    (nxn matrix of n columns) for the resulting scores
    base is the class interval size (default: 5)
    """
    for r1 in data:
        if r1 != 'patient':
            # convert continuous values to integer classes
            rater1 = (data[r1].round(decimals=2)*100).tolist()
            rater1 = list(map(int, rater1))
            rater1 = [base*math.floor(x/base) for x in rater1]
        for r2 in data:
            if r2 != 'patient':
                rater2 = (data[r2].round(decimals=2)*100).tolist()
                rater2 = list(map(int, rater2))
                rater2 = [base*math.floor(x/base) for x in rater2]
                # calculate the cohen kappa score 
                res.loc[r1, r2] = cohen_kappa_score(rater1, rater2)
    # minimal postprocessing before returning
    res = res.astype(float).round(decimals=2)
    return res


rows = list(candidates.keys())
res = pandas.DataFrame(data=None, index=rows, columns=rows)
res.drop(labels=['none', 'sim2'], axis=0, inplace=True)
res.drop(labels=['none', 'sim2'], axis=1, inplace=True)
res = irr(data, res, 5)
export = res.to_latex()
irrScores[met] = res
with open(met + '_irr.txt', 'w') as f: 
    f.write(export)

## Pre- and Postoperative Comparison

In [86]:
preop = data[data['patient'].str.endswith('1', na='nan')]
postop = data.mask(data['patient'].str.endswith('1', na='nan'))
postop = postop.dropna(axis=0)
postop.describe()

,gwa,aca,mav,sim,ekr,ise,rsa,aju,aka,mag,kch,sse
count,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.738660,0.743135,0.785955,0.785775,0.755778,0.770322,0.729388,0.782751,0.689845,0.606626,0.747090,0.743279
std,0.209099,0.220961,0.174050,0.167698,0.193444,0.195910,0.223253,0.159476,0.201113,0.282237,0.201669,0.202832
min,0.000000,0.000000,0.044492,0.052609,0.083428,0.000000,0.026506,0.090125,0.034239,0.000000,0.009784,0.000000
25%,0.696126,0.724369,0.734654,0.729638,0.714514,0.737244,0.697028,0.719186,0.620154,0.485485,0.697714,0.676401
50%,0.799151,0.813768,0.839071,0.840421,0.817484,0.836642,0.800622,0.839620,0.748423,0.703535,0.803403,0.801581
75%,0.876772,0.878664,0.893011,0.894934,0.878364,0.892474,0.885510,0.889776,0.826726,0.824166,0.877579,0.887709
max,0.946867,0.934203,0.950314,0.951563,0.948231,0.948391,0.943114,0.945424,0.921931,0.941268,0.937889,0.947616


In [91]:
# IRR for Pre- and Postoperative cases:
# create empty dataframe for correlation indices
res2 = pandas.DataFrame(data=None, index=rows, columns=rows)
temp = irr(preop, res2, base=5)
temp = temp.astype(float).round(decimals=2)
temp.drop(labels=['none', 'sim2', 'patient'], axis=0, inplace=True)
temp.drop(labels=['none', 'sim2'], axis=1, inplace=True)
print(temp)
export = temp.to_latex()
irrScores['preop'] = temp

      aju   aca   kch   ekr   aka   mag   sse   rsa   gwa   ise   mav   sim
aju  1.00  0.09  0.39  0.49  0.08  0.11  0.34  0.34  0.28  0.49  0.63  0.62
aca  0.09  1.00  0.21  0.19  0.13  0.09  0.27  0.14  0.20  0.16  0.12  0.13
kch  0.39  0.21  1.00  0.38  0.17  0.14  0.41  0.38  0.39  0.41  0.48  0.46
ekr  0.49  0.19  0.38  1.00  0.12  0.11  0.36  0.37  0.31  0.45  0.59  0.59
aka  0.08  0.13  0.17  0.12  1.00  0.07  0.18  0.22  0.22  0.09  0.10  0.10
mag  0.11  0.09  0.14  0.11  0.07  1.00  0.15  0.13  0.06  0.08  0.15  0.11
sse  0.34  0.27  0.41  0.36  0.18  0.15  1.00  0.32  0.38  0.32  0.35  0.34
rsa  0.34  0.14  0.38  0.37  0.22  0.13  0.32  1.00  0.22  0.34  0.38  0.39
gwa  0.28  0.20  0.39  0.31  0.22  0.06  0.38  0.22  1.00  0.29  0.33  0.32
ise  0.49  0.16  0.41  0.45  0.09  0.08  0.32  0.34  0.29  1.00  0.57  0.57
mav  0.63  0.12  0.48  0.59  0.10  0.15  0.35  0.38  0.33  0.57  1.00  0.91
sim  0.62  0.13  0.46  0.59  0.10  0.11  0.34  0.39  0.32  0.57  0.91  1.00


In [96]:
res3 = pandas.DataFrame(data=None, index=rows, columns=rows)
temp = irr(postop, res3, base=5)
temp = temp.astype(float).round(decimals=2)
temp.drop(labels=['none', 'sim2', 'patient'], axis=0, inplace=True)
temp.drop(labels=['none', 'sim2'], axis=1, inplace=True)
print(temp)
export = temp.to_latex()
irrScores['postop'] = temp

      aju   aca   kch   ekr   aka   mag   sse   rsa   gwa   ise   mav   sim
aju  1.00  0.21  0.31  0.49  0.09  0.02  0.29  0.45  0.19  0.52  0.43  0.51
aca  0.21  1.00  0.22  0.29  0.18  0.09  0.27  0.21  0.33  0.20  0.22  0.23
kch  0.31  0.22  1.00  0.32  0.14  0.09  0.33  0.43  0.43  0.33  0.37  0.36
ekr  0.49  0.29  0.32  1.00  0.16  0.08  0.32  0.39  0.26  0.40  0.50  0.55
aka  0.09  0.18  0.14  0.16  1.00  0.13  0.10  0.20  0.13  0.06  0.07  0.06
mag  0.02  0.09  0.09  0.08  0.13  1.00  0.12  0.08  0.12  0.03  0.05  0.05
sse  0.29  0.27  0.33  0.32  0.10  0.12  1.00  0.26  0.45  0.34  0.42  0.37
rsa  0.45  0.21  0.43  0.39  0.20  0.08  0.26  1.00  0.24  0.36  0.36  0.35
gwa  0.19  0.33  0.43  0.26  0.13  0.12  0.45  0.24  1.00  0.27  0.30  0.31
ise  0.52  0.20  0.33  0.40  0.06  0.03  0.34  0.36  0.27  1.00  0.60  0.62
mav  0.43  0.22  0.37  0.50  0.07  0.05  0.42  0.36  0.30  0.60  1.00  0.80
sim  0.51  0.23  0.36  0.55  0.06  0.05  0.37  0.35  0.31  0.62  0.80  1.00


## Dump Data to File 
And reload for testing purposes

In [97]:
# save all data to a pickle file
with open('irr.pkl', 'wb') as f:
        pickle.dump(irrScores, f, pickle.HIGHEST_PROTOCOL)

In [98]:
# load data again
with open('irr.pkl', 'rb') as f:
        scoresReloaded = pickle.load(f)

In [100]:
#test it
print(scoresReloaded['wt'])

aju        0.15
aca        1.00
kch        0.22
ekr        0.24
aka        0.15
mag        0.09
sse        0.27
rsa        0.17
gwa        0.26
ise        0.18
mav        0.17
sim        0.17
patient    0.27
Name: aca, dtype: float64


### Preop and Postop Data Collection

In [ ]:
#comparison of the mean values for pre- and post-operative scans
comp = pandas.concat([preop.mean(), postop.mean(), (postop.mean()- preop.mean()), preop.std(), postop.std()], axis=1, sort=True)
newname = {0:'Preop Mean', 
           1:'Postop Mean',
           2:'Difference',
           3: 'Preop STDEV',
           4: 'Postop STDEV'
          }
comp = comp.rename(columns=newname)
print('Comparison for region ' + met + ': \n', comp)
print('\nMean difference: ' + str(comp['Difference'].mean()))
print('\nMean difference stdev: ' + str(comp['Difference'].std()))
comp = comp.round(decimals=3)*100
tex = comp.to_latex()
with open('textable-'+met+'.txt', 'w') as f: 
    f.write(tex)
prepostplot = sb.boxplot(data=comp['Difference'], width=0.5, palette='spring')
# assemble comprehensive table
if met == 'wt':
    wtcomp = comp.mean()
if met == 'tc':
    tccomp = comp.mean()
if met == 'at':
    atcomp = comp.mean()
    
compressed = pandas.concat([wtcomp, tccomp, atcomp], axis=1, sort=False)
compressed = compressed.rename(columns={0: 'Whole Tumor', 1:'Tumor Core', 2:'Active Tumor'})
compressed = compressed.round(decimals=1)
print(compressed)
tex2 = compressed.to_latex()
with open('textable-complete.txt', 'w') as f: 
    f.write(tex2)

In [ ]:
if stupid:
    beauty = data[data.mav > 0.3]
    data = beauty
beauty.describe()

In [ ]:
#sort the indices by their mean value and reorder the data frame for descending plotting
m = data.mean()
m_sorted = m.sort_values(ascending=False)
print('Mean values sorted:')
pprint(m_sorted)
print('\nMedian sorted:')
pprint(data.median().sort_values(ascending=False))
print('\nStandard deviation sorted:')
pprint(data.std().sort_values(ascending=True))
indices = m_sorted.keys()
# order dataframe by mean
indices = indices.tolist()
wt_ordered = data[indices]

In [ ]:
# export all metrics to a table 
metrics = pandas.concat([m_sorted, data.median(), data.std()], axis=1, sort=True)
newname = {0:'Mean', 
           1:'Median',
           2:'Std-Dev'}
metrics = metrics.rename(columns=newname)
# round the values - who needs 8 decimals for such scores?!
metrics = metrics.round(decimals=3)
print(metrics)
metrics.to_csv('tables/'+met+'_table.csv')

In [ ]:
# fig = plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
bplot = sb.boxplot(data=wt_ordered, 
                 width=0.5,
                 palette="colorblind")
#bplot.axes.set_title('DICE Scores for whole tumor', fontsize=16)
bplot.set_xlabel('Algorithms', fontsize=8)
bplot.set_ylabel('Dice score', fontsize=8)
#plot a horizontal line at the max mean location
bplot.hlines(np.max(data.median()), -100, 1000, colors='r')
#save the plot
bplot.figure.savefig('plots/'+met+'_fullplot_thresh3.png', format=form, dpi=res)

In [ ]:
results = np.split(wt_ordered, [5], axis=1)
bplot1 = sb.boxplot(data=results[0], 
                 width=0.5,
                 palette="colorblind")
#bplot.axes.set_title('DICE Scores for whole tumor', fontsize=16)
bplot1.set_xlabel('Algorithms', fontsize=14)
bplot1.set_ylabel('Dice score', fontsize=14)

In [ ]:
bplot2 = sb.boxplot(data=results[1], 
                 width=0.5,
                 palette="colorblind")
#bplot.axes.set_title('DICE Scores for whole tumor', fontsize=16)
bplot2.set_xlabel('Algorithms', fontsize=14)
bplot2.set_ylabel('Dice score', fontsize=14)

In [ ]:
#save all of it
#bplot1.figure.savefig(met+'_boxplot1.png', format=form, dpi=res)
#bplot2.figure.savefig(met+'_boxplot2.png', format=form, dpi=res)
cl.savefig('clustermap.png', format=form, dpi=res)

In [ ]:
cl = sb.clustermap(wt_ordered, method='complete')

## Compound table for DICE

In [ ]:
data = pandas.read_csv('wt_dice_scores.csv')
wt_scores = preprocessor(data, 'whole')
tc_scores = preprocessor(pandas.read_csv('tc_dice_scores.csv'), 'core')
at_scores = preprocessor(pandas.read_csv('at_dice_scores.csv'), 'active')
wt_scores.describe()

In [ ]:
wt_m = wt_scores.median()
tc_m = tc_scores.median()
at_m = at_scores.median()
bobby = pandas.concat([wt_m, tc_m, at_m], axis=1, sort=True)
newkidz = {
    0 : 'WT',
    1 : 'TC',
    2 : 'AT'
}
bobby = bobby.rename(columns=newkidz)
bobby = bobby.round(decimals=3)*100
bobby.to_csv('tables/concisedice.csv')

## Specificity and Sensitivity Plotting

In [ ]:
# load data 
wt_spec = preprocessor(pandas.read_csv('specificity/wt_scores.csv'), 'wt-spec')
tc_spec = preprocessor(pandas.read_csv('specificity/tc_scores.csv'), 'tc-spec')
at_spec = preprocessor(pandas.read_csv('specificity/at_scores.csv'), 'at-spec')

wt_sens = preprocessor(pandas.read_csv('sensitivity/wt_scores.csv'), 'wt-sens')
tc_sens = preprocessor(pandas.read_csv('sensitivity/tc_scores.csv'), 'tc-sens')
at_sens = preprocessor(pandas.read_csv('sensitivity/at_scores.csv'), 'at-sens')


In [ ]:
wt_spec.describe()

In [ ]:
wt_sens.describe()

In [ ]:
# create Series for mean of specificity and sensitivity
wtsp_m = wt_spec.mean().round(decimals=3)
wtse_m = wt_sens.mean().round(decimals=3)
wtse_m

In [ ]:
# assemble Series to a single dataframe and rename the columns
wt_plt = pandas.concat([wtsp_m, wtse_m], axis=1, sort=True)
wt_plt = wt_plt.rename({0: 'Specificity', 1:'Sensitivity'}, axis=1)
# define some colors for the plots
colors = ['red','green','blue', 'aqua','gold','fuchsia', 'indigo','lavender','orange', 'sienna','teal','plum']
# plot sensitivity vs specificity 
ax = wt_plt.plot.scatter(x=0, y=1, c=colors, marker='s')
for i, txt in enumerate(wt_plt.index):
    ax.annotate(txt, (wt_plt.Specificity.iat[i],wt_plt.Sensitivity.iat[i]))
ax.grid(True)
ax.set_title('Whole Tumor Specificity vs Sensitivity')
ax.figure.savefig('wt_sesp.png', format=form, dpi=res)
#ax.annotate('aca', wt_plt[0][0], wt_plt[1][0])

In [ ]:
tcsp_m = tc_spec.mean().round(decimals=3)
tcse_m = tc_sens.mean().round(decimals=3)
tc_plt = pandas.concat([tcsp_m, tcse_m], axis=1, sort=True)
tc_plt = tc_plt.rename({0: 'Specificity', 1:'Sensitivity'}, axis=1)
colors = ['red','green','blue', 'aqua','gold','fuchsia', 'indigo','lavender','orange', 'sienna','teal','plum']
ax = tc_plt.plot.scatter(x=0, y=1, c=colors, marker='s')
for i, txt in enumerate(tc_plt.index):
    ax.annotate(txt, (tc_plt.Specificity.iat[i],tc_plt.Sensitivity.iat[i]))
ax.grid(True)
ax.set_title('Tumor Core Specificity vs Sensitivity')
ax.figure.savefig('tc_sesp.png', format=form, dpi=res)

In [ ]:
atsp_m = at_spec.mean().round(decimals=3)
atse_m = at_sens.mean().round(decimals=3)
at_plt = pandas.concat([atsp_m, atse_m], axis=1, sort=True)
at_plt = at_plt.rename({0: 'Specificity', 1:'Sensitivity'}, axis=1)
colors = ['red','green','blue', 'aqua','gold','fuchsia', 'indigo','lavender','orange', 'sienna','teal','plum']
ax = at_plt.plot.scatter(x=0, y=1, c=colors, marker='s')
for i, txt in enumerate(at_plt.index):
    ax.annotate(txt, (at_plt.Specificity.iat[i],at_plt.Sensitivity.iat[i]))
ax.grid(True)
ax.set_title('Active Tumor Specificity vs Sensitivity')
ax.figure.savefig('at_sesp.png', format=form, dpi=res)

In [ ]:
#once again but with MEDIAN 
atsp_m = at_spec.median().round(decimals=3)
atse_m = at_sens.median().round(decimals=3)
at_plt = pandas.concat([atsp_m, atse_m], axis=1, sort=True)
at_plt = at_plt.rename({0: 'Specificity', 1:'Sensitivity'}, axis=1)
colors = ['red','green','blue', 'aqua','gold','fuchsia', 'indigo','lavender','orange', 'sienna','teal','plum']
ax = at_plt.plot.scatter(x=0, y=1, c=colors, marker='s')
for i, txt in enumerate(at_plt.index):
    ax.annotate(txt, (at_plt.Specificity.iat[i],at_plt.Sensitivity.iat[i]))
ax.grid(True)
ax.set_title('Active Tumor Specificity vs Sensitivity')

In [ ]:
sens = at_plt.plot(y='Sensitivity', kind='bar')